In [1]:
%load_ext autoreload
%autoreload 2


# Extracting a dataset preprocessed and ready for decoding (and other population analyses)

### OR load a presaved DFallpa

In [3]:
import pandas as pd
path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa_Diego_230929_RULE_BASE_stroke.pkl"
DFallpa = pd.read_pickle(path)

# NEW MODULATION SCORE EUCLIDIAN

In [4]:
SAVEDIR_ANALYSIS = "/tmp"
import matplotlib.pyplot as plt
import os
import pandas as pd

In [9]:
# Plot only a subset of regions
bregions_plot = ["PMd_a", "preSMA_a"]

# Or plot all regions
# bregions_plot = None # TO plot all


### Get variables

In [10]:
# Method 1 - saved
from neuralmonkey.metadat.analy.anova_params import params_getter_euclidian_vars

question = "RULESW_ANBMCK_DIR_STROKE"
question = None
LIST_VAR, LIST_VARS_OTHERS, LIST_CONTEXT, LIST_PRUNE_MIN_N_LEVS, LIST_FILTDICT = params_getter_euclidian_vars(question)

... testing:  RULE_ROWCOL_STROKE
... testing:  RULE_DIR_STROKE
... testing:  RULE_ANBMCK_STROKE
... testing:  RULESW_ANBMCK_DIR_STROKE
... testing:  RULE_COLRANK_STROKE
... testing:  RULESW_ANBMCK_COLRANK_STROKE
PAssed all tests!


In [25]:
# Method 2 - hand enter (for testing)

# Variable of effect
LIST_VAR = [
    "stroke_index", # preSMA reflect stroke index no matter the shape
]

# Varaibles to condition on
LIST_VARS_OTHERS = [
    ["epoch", "shape"], # .
    ]

# For purposes of testing generalization acros contexts. Leave as None to skip
# LIST_CONTEXT = [
#     {"same":["epoch"], "diff":["shape"]},
# ]
LIST_CONTEXT = [
    None
]

# N min levels needed to keep contexts
LIST_PRUNE_MIN_N_LEVS = [
    2 for _ in range(len(LIST_VAR))
]

# Optional: filter data (throw out trials that fail filter)
LIST_FILTDICT = [
    None
]
# LIST_FILTDICT = [
#     {"stroke_index": list(range(1, 10, 1))},
# ]


In [32]:
from neuralmonkey.analyses.decode_good import euclidian_distance_compute

# PARAMS: data represnetation
SHUFFLE = False
extra_dimred_method = "umap"
umap_n_neighbors = 40
NPCS_KEEP = 10

# PARAMS (Remove plots to speed up)
PLOT = False # plot raw
PLOT_MASKS = False # Plot context masks (can be slow!)
nmin_trials_per_lev = 5 # to prune data
PLOT_STATE_SPACE = True # Scatter plot of neural data -- useful!
twind = (-0.1, 0.1) # relative to event
tbin_dur = 0.1 # windowing
tbin_slice = tbin_dur

# RUN
list_dfres = []
for i, row in DFallpa.iterrows():
    br = row["bregion"]
    tw = row["twind"]
    ev = row["event"]
    PA = row["pa"]
    
    if bregions_plot is not None:
        if br not in bregions_plot:
            continue

    savedir = f"{SAVEDIR_ANALYSIS}/{br}"
    os.makedirs(savedir, exist_ok=True)

    dfres = euclidian_distance_compute(PA, LIST_VAR, LIST_VARS_OTHERS, PLOT, PLOT_MASKS,
                               twind, tbin_dur, tbin_slice, savedir, PLOT_STATE_SPACE=PLOT_STATE_SPACE,
                                       nmin_trials_per_lev=nmin_trials_per_lev, LIST_CONTEXT=LIST_CONTEXT,
                                       LIST_FILTDICT=LIST_FILTDICT, NPCS_KEEP=NPCS_KEEP, extra_dimred_method=extra_dimred_method, 
                                       umap_n_neighbors = umap_n_neighbors, LIST_PRUNE_MIN_N_LEVS=LIST_PRUNE_MIN_N_LEVS)
    plt.close("all")
    if len(dfres)>0:
        dfres["bregion"] = br
        dfres["twind"] = [tw for _ in range(len(dfres))]
        dfres["event"] = ev
        list_dfres.append(dfres)
        
DFRES = pd.concat(list_dfres).reset_index(drop=True)

UMAP, Using this n_neighbors: 40 , nsamp = 2082
DIST_NULL_50 2.816599328191034
DIST_NULL_95 5.390514893211162
DIST_NULL_98 5.875625654669852
RUNNING:  0 stroke_index  --  ['epoch', 'shape']
  Pruning for this var adn conjunction. Original length: 2082 , pruned length: 2082
  Final Scalar data (trial, dims): (2082, 2)
fig: /tmp/PMd_a/color=stroke_index-sub=epoch|shape-dims=(0, 1)-0.pdf
Saving ...  /tmp/PMd_a/color=stroke_index-sub=epoch|shape-dims=(0, 1)-0.pdf
UMAP, Using this n_neighbors: 40 , nsamp = 2082
DIST_NULL_50 3.158915345638853
DIST_NULL_95 6.7598049509793485
DIST_NULL_98 7.332680600460303
RUNNING:  0 stroke_index  --  ['epoch', 'shape']
  Pruning for this var adn conjunction. Original length: 2082 , pruned length: 2082
  Final Scalar data (trial, dims): (2082, 2)
fig: /tmp/preSMA_a/color=stroke_index-sub=epoch|shape-dims=(0, 1)-0.pdf
Saving ...  /tmp/preSMA_a/color=stroke_index-sub=epoch|shape-dims=(0, 1)-0.pdf


### Save and plot results

In [33]:
SAVEDIR = f"{SAVEDIR_ANALYSIS}/RESULTS"
import os
from pythonlib.tools.pandastools import append_col_with_grp_index
from neuralmonkey.scripts.analy_euclidian_dist_pop_script import plot_all_results
os.makedirs(SAVEDIR, exist_ok=True)

In [34]:
# SAVE
path = f"{SAVEDIR}/DFRES.pkl"
pd.to_pickle(DFRES, path)
print("Saved to: ", path)


Saved to:  /tmp/RESULTS/DFRES.pkl


In [35]:
############ PLOTS
plot_all_results(DFRES, SAVEDIR)

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
/home/lucas/code/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
/home/lucas/code/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/

...  /tmp/RESULTS/FIGURES/histograms-dat_level=distr


/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


Plotting specific conjucntions heatmaps ... 
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=diff|same-shuffled=False-dat_level=distr
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=same|diff-shuffled=False-dat_level=distr
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=any|diff-shuffled=False-dat_level=distr
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=diff|diff-shuffled=False-dat_level=distr
...  /tmp/RESULTS/FIGURES/histograms-dat_level=pts


/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


Plotting specific conjucntions heatmaps ... 
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=diff|same-shuffled=False-dat_level=pts
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=same|same-shuffled=False-dat_level=pts
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=same|diff-shuffled=False-dat_level=pts
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=any|diff-shuffled=False-dat_level=pts
...  /tmp/RESULTS/FIGURES/each_conjunction-effect_context=diff|diff-shuffled=False-dat_level=pts
